In [ ]:
import octis
import pandas as pd

In [ ]:
newdf = pd.read_csv('tweets_filt.csv')

In [ ]:
newdf.head()

,username,tweet,likes_count
0,pabloandrespozo,#endlessbodegon #NYC #NFTCommunity #NFT_NYC @N...,0
1,fionaaboud,@FerCaggianoArt rockin get awesome knowledge a...,0
2,jeffjoz,@Bupolo_ @ThetaDrop @Theta_Network @SonyElectr...,0
3,lanell97977606,🗽 This year's @NFT_NYC will be bigger than eve...,0
4,artangelsla,@LindseyPriceArt @NFT_NYC 👏💙😇,0


In [ ]:
# Load the regular expression library
import re
# Remove punctuation
newdf['tweet_processed'] = \
newdf['tweet'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
newdf['tweet_processed'] = \
newdf['tweet_processed'].map(lambda x: x.lower())

newdf['tweet_processed'].head()

0    #endlessbodegon #nyc #nftcommunity #nft_nyc @n...
1    @fercaggianoart rockin get awesome knowledge a...
2    @bupolo_ @thetadrop @theta_network @sonyelectr...
3    🗽 this year's @nft_nyc will be bigger than eve...
4                        @lindseypriceart @nft_nyc 👏💙😇
Name: tweet_processed, dtype: object

In [ ]:
#reuse LDA code from previous project

import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = newdf.tweet_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['endlessbodegon', 'nyc', 'nftcommunity', 'nft_nyc', 'nft_nyc', 'https', 'tco', 'nfc', 'nvj']


In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1)]


In [ ]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.097*"nft_nyc" + 0.037*"superchiefnft" + 0.028*"timepieces" + '
  '0.018*"rugradio" + 0.017*"congrats" + 0.015*"congratulations" + '
  '0.014*"unknownco" + 0.014*"vizmesh" + 0.014*"farokh" + '
  '0.014*"johnknopfphotos"'),
 (1,
  '0.177*"nft_nyc" + 0.049*"projectgalaxyhq" + 0.019*"bluugu" + 0.013*"cool" + '
  '0.011*"nice" + 0.008*"axieinfinity" + 0.008*"thesandboxgame" + '
  '0.007*"great" + 0.005*"wait" + 0.005*"mhmarquisnyc"'),
 (2,
  '0.087*"nft_nyc" + 0.016*"cozomomedici" + 0.013*"nickelodeon_nft" + '
  '0.011*"nonfungiblefilm" + 0.010*"osf_nft" + 0.009*"usmverse" + '
  '0.009*"frenchmontana" + 0.008*"mtg_usm" + 0.008*"rektguynft" + 0.006*"wax"'),
 (3,
  '0.188*"nft_nyc" + 0.036*"gm" + 0.022*"good" + 0.014*"ded" + 0.010*"web" + '
  '0.009*"project" + 0.007*"enjoy" + 0.007*"nicolaarmento" + '
  '0.007*"matthewmorrocco" + 0.007*"wearetopramen"'),
 (4,
  '0.089*"nft_nyc" + 0.022*"lfg" + 0.021*"https" + 0.021*"tco" + '
  '0.015*"nftnyc" + 0.013*"nft" + 0.012*"flow_blockchain"